# 로지스틱 회귀
- 로지스틱 회귀 알고리즘 학습 및 이진 분류 문제에서 클래스 확률 예측

k-최근접 이웃은 주변 이웃을 찾아줌 &rarr; 이웃의 클래스 비율을 확률이라고 출력
- k-최근접 이웃 분류기도 위 아이디어와 동일한 방식으로 클래스 확률을 계산하여 제공

In [ ]:
# 데이터 준비
# 데이터 출처 : https://bit.ly/fish_csv_data
import pandas as pd

fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()

데이터프레임에서 맨 왼쪽 숫자는 행 번호(판다스의 인덱스)

맨 위에 쓰여진 내용들은 열 제목

판다스는 csv파일의 첫 줄을 자동으로 인식해 열 제목으로 만들어 줌

In [ ]:
#  어떤 생선이 존재하는지 고유한 값을 추출
# 판다스의 unique()함수 사용
print(pd.unique(fish['Species']))

In [ ]:
# 데이터프레임에서 Species열을 타깃으로 만들고 나머지 5개 열은 입력 데이터로 사용
fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()
print(fish_input[:5])

In [ ]:
# 타깃 데이터 생성
fish_target = fish['Species'].to_numpy()

In [ ]:
# 데이터를 훈련 세트와 테스트 세트로 분리
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state=42)

In [ ]:
# 사이킷런의 StandardScaler 사용하여 훈련 세트와 테스트 세트 표준화
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [ ]:
# KNeighborClassfier 클래스 객체를 만들고 훈련 세트로 모델 훈련 및 점수 확인
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)
print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))

타깃 데이터에 2개 이상의 클래스가 포함된 문제를 ***다중 분류***라고 함

이진 분류를 사용했을 때 양성, 음성 클래스를 1,0으로 표기하는 것 처럼

다중 분류에서도 타깃값을 숫자로 바구어 입력 가능
- 사이킷런에서는 편리하게 문자열로 된 타깃값 그대로 사용 가능
- 다만 사이킷런에서 자동으로 순서가 알파벳 순으로 정렬됨

KNeighborsClassfier에서 정렬된 타깃값은 classes_속성에 저장됨

In [ ]:
print(kn.classes_)

In [ ]:
# predict()메서드는 타깃값으로 예측을 출력
print(kn.predict(test_scaled[:5]))

In [ ]:
# predict_proba() 메서드로 클래스별 확률값을 반환
# 기본적으로 소수점 첫째 자리에서 반올림
# decimals 매개변수로 유지할 소수점 아래 자릿수 지정 가능
import numpy as np

proba = kn.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=4))

In [ ]:
# kneighbors 메서드의 입력은 2차원 배열이어야 함
# 넘파이 배열의 슬라이싱 연산자 사용
# 슬라이싱 연산자는 하나의 샘플만 선택해도 항상 2차원 배열 만들어짐
# 4번 째 샘플의 최근접 이웃 클래스 확인
distances, indexes = kn.kneighbors(test_scaled[3:4])
print(train_target[indexes])

***로지스틱 회귀***는 이름은 회귀지만 분류모델

선형 회귀와 동일하게 선형 방정식 학습

$
z = a * (Weight) + b * (Lenght) + c * (Diagond) + d * (Height) + e * (Width) + f
$

여기서 a,b,c,d,e는 가중치 혹은 계수

특성은 늘어났지만 다중 회귀를 위한 선형 방정식과 같다

z는 어떤 값도 가능
- 하지만 확률이 되려면 0~1(또는 0~100%)사이 값이 되어야 함
- z가 아주 큰 음수일 때 0, 아주 큰 양수일 때 1이 되도록 바꿔야함
- ***시그모이드 함수***(***로지스틱 함수***)를 사용하면 가능

In [ ]:
# 넘파이를 사용해 그래프 생성
# 지수 함수 계산은 np.exp() 함수 사용
import numpy as np
import matplotlib.pyplot as plt

# -5와 5 사이에 0.1 간격으로 z 생성
z = np.arange(-5, 5, 0.1)
# z 위치마다 시그모이드 함수 계산
phi = 1 / (1 + np.exp(-z))
plt.plot(z, phi)
plt.xlabel('z')
plt.ylabel('phi')
plt.show()

In [ ]:
# 로지스틱 회귀로 이진 분류 수행
# 넘파이 배열은 True, False 값을 전달하여 행을 선택 가능 -> 불리언 인덱싱
char_arr = np.array(['A', 'B', 'C', 'D', 'E'])
print(char_arr[[True, False, True, False, False]])

In [ ]:
# 위의 예시와 같이 Bream과 Smelt의 행만 골라내기
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]

In [ ]:
# LogisticRegression 사용하여 로지스틱 회귀 모델 생성
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)

In [ ]:
# 훈련한 모델을 사용하여 train_bream_smelt에 있는 처음 5개 샘플 예측
print(lr.predict(train_bream_smelt[:5]))

In [ ]:
# 예측 확률 출력
print(lr.predict_proba(train_bream_smelt[:5]))

In [ ]:
# Bream과 Smelt 중 양성 음성 판별
# 이진 분류에서 첫 번째 열이 음성(0)클래스, 두 번째 열이 양성(1)클래스
print(lr.classes_)

In [ ]:
# 로지스틱 회귀가 학습한 계수 확인
print(lr.coef_, lr.intercept_)

In [ ]:
# LogisticRegression 모델로 z 값 계산
# decision_function()메서드로 z값 출력 가능
decisions = lr.decision_function(train_bream_smelt[:5])
print(decisions)

In [ ]:
# 위의 z 값을 시그모이드 함수에 통과 시키면 확률을 얻을 수 있음
# 파이썬의 scipy 라이브러리에도 시그모이드 함수가 있음 -> expit()
from scipy.special import expit
print(expit(decisions))

# predict_proba()의 두 번째 열의 값과 동일한 것 확인 가능
# 즉, decision_function() 메서드는 양성 클래스에 대한 z 값 반환

로지스틱 회귀로 다중 분류 수행하기
- LogisticRegression 클래스는 기본적으로 반복적인 알고리즘 사용
- max_iter 매개변수에서 반복 횟수 지정 가능(기본값은 100)
- 기본적으로 릿지 회귀와 같이 계수의 제곱을 규제
- 규제를 제어하는 매개변수는 C
- C는 alpha와 반대로 작을수록 규제가 커짐(기본값은 1)

In [ ]:
# LogisticRegression 클래스로 다중 분류 모델을 훈련하는 코드 작성
lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled, train_target)
print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

In [ ]:
# 테스트 세트의 처음 5개 샘플에 대한 예측
print(lr.predict(test_scaled[:5]))

In [ ]:
# 테스트 세트의 처음 5개 샘플에 대한 예측 확률을 출력
# 출력을 간소화 하기 위해 소수점 네 번째 자리에서 반올림
proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))

In [ ]:
# 모델의 클래스 정보 확인
print(lr.classes_)

In [ ]:
# coef_, intercept_의 크기 출력
print(lr.coef_.shape, lr.intercept_.shape)

이 데이터는 5개의 특성을 사용하므로 coef_배열의 열은 5개, 행은 7개, intercept_도 7개

이진 분류에서 보았던 z를 7개 계산한다는 의미

다중 분류는 클래스마다 z 값을 하나씩 계산, 가장 높은 z 값을 출력하는 클래스가 예측 클래스가 됨

다중 분류는 ***소프트맥스*** 함수를 사용하여 7개의 z값을 확률로 변환

$
e_sum = e^z1 + e^z2 + e^z3 + e^z4 + e^z5 + e^z6 + e^z7
$

그 다음 $e^z1$ ~ $e^z7$을 각각 e_sum으로 나누어 줌


s1 = $e^z1$ / e_sum, .... s7 = $e^z7$ / e_sum



In [ ]:
# 이진 분류에서처럼 decision_function() 메서드로 z1~z7까지의 값을 구함
# 소프트맥스 함수를 사용해 확률로 변경
# 테스트 세트의 처음 5개 샘플에 대한 값 구하기
decision = lr.decision_function(test_scaled[:5])
print(np.round(decision, decimals=2))

In [ ]:
# scipy.special 아래에 softmax() 함수를 호출하여 사용 가능
from scipy.special import softmax
proba = softmax(decision, axis=1)
print(np.round(proba, decimals=3))

# axis 매개변수는 소프트맥스를 계산할 축을 지정, axis=1은 각 행, 즉 각 샘플에 대해 소프트 맥스를 계산
# 앞에서 구한 proba 배열과 값이 일치